In [ ]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib notebook

In [ ]:
radii = [1.0, 0.5] # Outer is always first
velocities = [1.0, 2.0]

In [ ]:
def draw_earth():
    x0 = np.linspace(-1,1,500)
    for r in radii:
        y = np.sqrt(r*r - x0*x0)
        plt.plot(x0,y,'k-')
        plt.plot(x0,-y,'k-')

In [ ]:
fig = plt.figure(figsize=(6,6))
draw_earth()